In [54]:
# pip install matplotlib

In [55]:
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt

### Data Import

In [56]:
# ETF
etf = pd.read_csv('./etf_month_JY.csv')  # % 수익률
etf.columns = ['DATE', 'SPY', 'LQD', 'IEF']
# etf.set_index('DATE', inplace=True)

# Risk Free
rf = pd.read_csv('./rf_JY.csv')
# rf.set_index('DATE', inplace=True)

# Risk Premium
# credit01 = pd.read_csv('./credit01_JY.csv')
# credit02 = pd.read_csv('./credit02_JY.csv')
credit03 = pd.read_csv('./credit03_JY.csv')
# credit03.set_index('DATE', inplace=True)

# term01 = pd.read_csv('./term01_JY.csv')
# term02 = pd.read_csv('./term02_JY.csv')
term03 = pd.read_csv('./term03_JY.csv')
# term03.set_index('DATE', inplace=True)

equity01 = pd.read_csv('./equity01_JY.csv')
# equity01.set_index('DATE', inplace=True)
# equity02 = pd.read_csv('./equity02_JY.csv')

In [57]:
etf.iloc[:,1:] /= 100
etf#.iloc[:,1:].mean()*10000  # 소수점 단위 완료

,DATE,SPY,LQD,IEF
0,2003-01,-0.055013,0.013695,0.008255
1,2003-02,-0.013479,0.019799,0.022040
2,2003-03,0.002140,0.001208,-0.003799
3,2003-04,0.084612,0.023578,0.001380
4,2003-05,0.054836,0.039052,0.037909
...,...,...,...,...
235,2022-08,-0.040802,-0.044291,-0.038538
236,2022-09,-0.092446,-0.059798,-0.047350
237,2022-10,0.081275,-0.007453,-0.014540
238,2022-11,0.055592,0.066477,0.036123


In [58]:
equity01.iloc[:,1] /= 12
equity01  # 연 수익률을 월 환산

,DATE,Return
0,2003-02,-0.020256
1,2003-03,0.005185
2,2003-04,0.077745
3,2003-05,0.047925
4,2003-06,0.008544
...,...,...
222,2021-08,0.027921
223,2021-09,-0.048715
224,2021-10,0.067825
225,2021-11,-0.009633


In [59]:
rf.iloc[:,1] /= 12
rf  # 연 수익률을 월 환산

,DATE,RF
0,2003-01,0.008333
1,2003-02,0.007500
2,2003-03,0.008333
3,2003-04,0.008333
4,2003-05,0.007500
...,...,...
221,2021-06,0.000000
222,2021-07,0.000000
223,2021-08,0.000000
224,2021-09,0.000000


In [60]:
credit03.iloc[:,1] /= 12
credit03

,DATE,Return
0,2003-01,0.004146
1,2003-02,0.003973
2,2003-03,0.003951
3,2003-04,0.003852
4,2003-05,0.003472
...,...,...
223,2021-08,0.002080
224,2021-09,0.002050
225,2021-10,0.002188
226,2021-11,0.002145


In [61]:
term03.iloc[:,1] /= 12
term03

,DATE,Return
0,2003-01,0.002383
1,2003-02,0.002264
2,2003-03,0.002213
3,2003-04,0.002338
4,2003-05,0.002067
...,...,...
223,2021-08,0.001024
224,2021-09,0.001111
225,2021-10,0.001275
226,2021-11,0.001257


In [62]:
# Regime : List
recovery = ['2003-04', '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', '2009-05', '2009-06', '2009-07', '2009-08', '2009-09', '2009-10', '2009-11', '2009-12', '2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-07', '2010-08', '2010-09', '2010-10', '2011-06', '2011-12', '2012-01', '2012-06', '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2015-10', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09', '2016-11', '2016-12', '2017-01', '2017-02', '2019-02', '2019-03', '2019-04', '2019-06', '2019-09', '2019-10', '2019-11', '2019-12', '2020-11']
expansion = ['2003-10', '2003-11', '2003-12', '2004-01', '2004-02', '2004-03', '2004-04', '2004-05', '2004-06', '2004-07', '2004-08', '2004-09', '2004-10', '2004-11', '2004-12', '2005-01', '2005-02', '2005-03', '2005-04', '2005-05', '2005-06', '2005-07', '2005-08', '2005-09', '2005-11', '2005-12', '2006-01', '2006-02', '2006-03', '2006-04', '2006-08', '2006-09', '2006-10', '2006-11', '2006-12', '2007-01', '2008-05', '2010-11', '2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2012-02', '2012-03', '2012-04', '2012-12', '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-09', '2013-10', '2013-11', '2013-12', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-08', '2014-10', '2014-11', '2015-02', '2015-04', '2015-05', '2015-07', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-07', '2018-08', '2018-09', '2020-12', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10']
slowdown = ['2005-10', '2006-05', '2006-06', '2006-07', '2007-02', '2007-03', '2007-04', '2007-05', '2007-06', '2007-07', '2007-08', '2007-09', '2007-10', '2007-11', '2007-12', '2008-01', '2008-02', '2008-03', '2008-04', '2008-06', '2013-08', '2014-01', '2014-07', '2014-09', '2014-12', '2015-01', '2015-03', '2015-06', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-10', '2018-11', '2018-12', '2021-01', '2021-02']
contraction = ['2003-01', '2003-02', '2003-03', '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12', '2009-01', '2009-02', '2009-03', '2009-04', '2010-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2012-05', '2015-08', '2015-09', '2015-11', '2015-12', '2016-01', '2016-02', '2016-10', '2019-01', '2019-05', '2019-07', '2019-08', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10']

In [63]:
# Buy and Hold : List


start_date = '2003-01-01'
end_date = '2021-10-31'

bh = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m-%d').tolist()

In [64]:
bh

['2003-01-31',
 '2003-02-28',
 '2003-03-31',
 '2003-04-30',
 '2003-05-31',
 '2003-06-30',
 '2003-07-31',
 '2003-08-31',
 '2003-09-30',
 '2003-10-31',
 '2003-11-30',
 '2003-12-31',
 '2004-01-31',
 '2004-02-29',
 '2004-03-31',
 '2004-04-30',
 '2004-05-31',
 '2004-06-30',
 '2004-07-31',
 '2004-08-31',
 '2004-09-30',
 '2004-10-31',
 '2004-11-30',
 '2004-12-31',
 '2005-01-31',
 '2005-02-28',
 '2005-03-31',
 '2005-04-30',
 '2005-05-31',
 '2005-06-30',
 '2005-07-31',
 '2005-08-31',
 '2005-09-30',
 '2005-10-31',
 '2005-11-30',
 '2005-12-31',
 '2006-01-31',
 '2006-02-28',
 '2006-03-31',
 '2006-04-30',
 '2006-05-31',
 '2006-06-30',
 '2006-07-31',
 '2006-08-31',
 '2006-09-30',
 '2006-10-31',
 '2006-11-30',
 '2006-12-31',
 '2007-01-31',
 '2007-02-28',
 '2007-03-31',
 '2007-04-30',
 '2007-05-31',
 '2007-06-30',
 '2007-07-31',
 '2007-08-31',
 '2007-09-30',
 '2007-10-31',
 '2007-11-30',
 '2007-12-31',
 '2008-01-31',
 '2008-02-29',
 '2008-03-31',
 '2008-04-30',
 '2008-05-31',
 '2008-06-30',
 '2008-07-

### Recovery - SPY

In [65]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data01 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data01 = data01.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data01 = data01.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data01 = data01.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data01 = data01.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data01 = data01.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data01 = data01.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data01 = data01.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data01 = data01.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data01.columns = ['DATE', 'SPY', 'rf', 'credit03', 'term03', 'equity01']
data01['DATE'] = pd.to_datetime(data01['DATE'])
data01['DATE'] = data01['DATE'].dt.strftime('%Y-%m')
data01.set_index('DATE', inplace=True)

# Assuming recovery_dates is a list of datetime objects representing the recovery dates
recovery_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in recovery] 
recovery_dates

# recovery 컬럼 추가
data01['recovery'] = data01.index.map(lambda x: 1 if x in recovery_dates else 0)
# data01['recovery'].unique()

# recovery 국면만 슬라이싱
data01 = data01[data01['recovery'] == 1]
data01

# Selecting relevant columns for the regression
X = data01[['credit03', 'term03', 'equity01']]
y = data01['SPY']-data01['rf']

# # Adding a constant to the independent variables
X = sm.add_constant(X)

# Performing the OLS regression
model01 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model01.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     2786.
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           1.02e-54
Time:                        23:11:39   Log-Likelihood:                 239.60
No. Observations:                  53   AIC:                            -471.2
Df Residuals:                      49   BIC:                            -463.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0236      0.003     -9.319      0.000      -0.029      -0.019
credit03      14.0045      1.970      7.110      0.000      10.046      17.963
term03       -11.3373      2.191     -5.174      0.000     -15.740      -6.934
equity01       0.9942      0.011     89.425      0.000       0.972       1.017
==============================================================================
Omnibus:                        1.143   Durbin-Watson:                   1.094
Prob(Omnibus):                  0.565   Jarque-Bera (JB):                0.579
Skew:                          -0.231   Prob(JB):                        0.749
Kurtosis:                       3.220   Cond. No.                     7.80e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.8e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [66]:
data01

,SPY,rf,credit03,term03,equity01,recovery
DATE,,,,,,
2003-04,0.084612,0.008333,0.003852,0.002338,0.077745,1
2003-05,0.054836,0.007500,0.003472,0.002067,0.047925,1
2003-06,0.010652,0.008333,0.003351,0.001998,0.008544,1
2003-07,0.018028,0.005833,0.003727,0.002546,0.012911,1
2003-08,0.020626,0.005833,0.004086,0.002897,0.014169,1
2003-09,-0.010906,0.006667,0.003994,0.002766,-0.015506,1
2009-05,0.058453,0.000000,0.004463,0.002597,0.050337,1
2009-06,-0.000655,0.000833,0.004543,0.002953,-0.002906,1
2009-07,0.074606,0.000833,0.004354,0.002816,0.071173,1


### Recovery - LQD

In [67]:
# 데이터 병합

etf_month_LQD = etf[['DATE', 'LQD']]
data02 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data02 = data02.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data02 = data02.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data02 = data02.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data02 = data02.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data02 = data02.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data02 = data02.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data02 = data02.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data02 = data02.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data02.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data02['DATE'] = pd.to_datetime(data02['DATE'])
data02['DATE'] = data02['DATE'].dt.strftime('%Y-%m')
data02.set_index('DATE', inplace=True)

# Assuming recovery_dates is a list of datetime objects representing the recovery dates
recovery_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in recovery] 
recovery_dates

# recovery 컬럼 추가
data02['recovery'] = data02.index.map(lambda x: 1 if x in recovery_dates else 0)

# recovery 국면만 슬라이싱
data02 = data02[data02['recovery'] == 1]
data02

# Selecting relevant columns for the regression
X = data02[['credit03', 'term03', 'equity01']]
y = data02['LQD']-data02['rf']

# # Adding a constant to the independent variables
X = sm.add_constant(X)

# Performing the OLS regression
model02 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model02.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     3.382
Date:                Thu, 23 Nov 2023   Prob (F-statistic):             0.0254
Time:                        23:11:40   Log-Likelihood:                 137.51
No. Observations:                  53   AIC:                            -267.0
Df Residuals:                      49   BIC:                            -259.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0395      0.017     -2.271      0.028      -0.074      -0.005
credit03      29.7456     13.517      2.201      0.033       2.582      56.909
term03       -28.3020     15.037     -1.882      0.066     -58.520       1.916
equity01       0.1247      0.076      1.635      0.109      -0.029       0.278
==============================================================================
Omnibus:                       10.296   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               12.112
Skew:                          -0.722   Prob(JB):                      0.00234
Kurtosis:                       4.843   Cond. No.                     7.80e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.8e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Recovery - IEF

In [68]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data03 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data03 = data03.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data03 = data03.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data03 = data03.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data03 = data03.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data03 = data03.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data03 = data03.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data03 = data03.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data03 = data03.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data03.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data03['DATE'] = pd.to_datetime(data03['DATE'])
data03['DATE'] = data03['DATE'].dt.strftime('%Y-%m')
data03.set_index('DATE', inplace=True)

# Assuming recovery_dates is a list of datetime objects representing the recovery dates
recovery_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in recovery] 
recovery_dates

# recovery 컬럼 추가
data03['recovery'] = data03.index.map(lambda x: 1 if x in recovery_dates else 0)

# recovery 국면만 슬라이싱
data03 = data03[data03['recovery'] == 1]
data03

# Selecting relevant columns for the regression
X = data03[['credit03', 'term03', 'equity01']]
y = data03['IEF'] - data03['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model03 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model03.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     2.283
Date:                Thu, 23 Nov 2023   Prob (F-statistic):             0.0907
Time:                        23:11:40   Log-Likelihood:                 139.11
No. Observations:                  53   AIC:                            -270.2
Df Residuals:                      49   BIC:                            -262.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0233      0.017     -1.380      0.174      -0.057       0.011
credit03      14.7521     13.114      1.125      0.266     -11.602      41.107
term03       -12.0539     14.589     -0.826      0.413     -41.371      17.263
equity01      -0.1213      0.074     -1.639      0.108      -0.270       0.027
==============================================================================
Omnibus:                       10.163   Durbin-Watson:                   2.302
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               11.125
Skew:                          -0.762   Prob(JB):                      0.00384
Kurtosis:                       4.647   Cond. No.                     7.80e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.8e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Expansion - SPY

In [69]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data04 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data04 = data04.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data04 = data04.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data04 = data04.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data04 = data04.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data04 = data04.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data04 = data04.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data04 = data04.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data04 = data04.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data04.columns = ['DATE', 'SPY', 'rf', 'credit03','term03', 'equity01']
data04['DATE'] = pd.to_datetime(data04['DATE'])
data04['DATE'] = data04['DATE'].dt.strftime('%Y-%m')
data04.set_index('DATE', inplace=True)

# Assuming expansion_dates is a list of datetime objects representing the expansion dates
expansion_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in expansion] 
expansion_dates

# expansion 컬럼 추가
data04['expansion'] = data04.index.map(lambda x: 1 if x in expansion_dates else 0)

# expansion 국면만 슬라이싱
data04 = data04[data04['expansion'] == 1]
data04

# Selecting relevant columns for the regression
X = data04[['credit03', 'term03', 'equity01']]
y = data04['SPY'] - data04['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model04 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model04.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.962
Method:                 Least Squares   F-statistic:                     777.4
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           1.07e-63
Time:                        23:11:40   Log-Likelihood:                 364.83
No. Observations:                  93   AIC:                            -721.7
Df Residuals:                      89   BIC:                            -711.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0334      0.002    -16.359      0.000      -0.038      -0.029
credit03      19.4925      2.272      8.580      0.000      14.979      24.006
term03       -15.3591      2.876     -5.341      0.000     -21.073      -9.645
equity01       1.0313      0.024     42.352      0.000       0.983       1.080
==============================================================================
Omnibus:                        2.756   Durbin-Watson:                   0.444
Prob(Omnibus):                  0.252   Jarque-Bera (JB):                2.219
Skew:                           0.367   Prob(JB):                        0.330
Kurtosis:                       3.188   Cond. No.                     7.19e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.19e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Expansion - LQD

In [70]:
# 데이터 병합
etf_month_LQD = etf[['DATE', 'LQD']]
data05 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data05 = data05.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data05 = data05.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data05 = data05.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data05 = data05.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data05 = data05.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data05 = data05.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data05 = data05.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data05 = data05.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data05.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data05['DATE'] = pd.to_datetime(data05['DATE'])
data05['DATE'] = data05['DATE'].dt.strftime('%Y-%m')
data05.set_index('DATE', inplace=True)

# Assuming expansion_dates is a list of datetime objects representing the expansion dates
expansion_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in expansion] 
expansion_dates

# expansion 컬럼 추가
data05['expansion'] = data05.index.map(lambda x: 1 if x in expansion_dates else 0)

# expansion 국면만 슬라이싱
data05 = data05[data05['expansion'] == 1]
data05

# Selecting relevant columns for the regression
X = data05[['credit03', 'term03', 'equity01']]
y = data05['LQD'] - data05['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model05 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model05.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.292
Model:                            OLS   Adj. R-squared:                  0.269
Method:                 Least Squares   F-statistic:                     12.26
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           8.66e-07
Time:                        23:11:40   Log-Likelihood:                 263.39
No. Observations:                  93   AIC:                            -518.8
Df Residuals:                      89   BIC:                            -508.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0328      0.006     -5.397      0.000      -0.045      -0.021
credit03      15.1095      6.762      2.234      0.028       1.674      28.545
term03        -9.5691      8.560     -1.118      0.267     -26.577       7.439
equity01       0.1021      0.072      1.409      0.162      -0.042       0.246
==============================================================================
Omnibus:                        8.522   Durbin-Watson:                   1.561
Prob(Omnibus):                  0.014   Jarque-Bera (JB):                8.378
Skew:                          -0.721   Prob(JB):                       0.0152
Kurtosis:                       3.285   Cond. No.                     7.19e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.19e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Expansion - IEF

In [71]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data06 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data06 = data06.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data06 = data06.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data06 = data06.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data06 = data06.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data06 = data06.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data06 = data06.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data06 = data06.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data06 = data06.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data06.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data06['DATE'] = pd.to_datetime(data06['DATE'])
data06['DATE'] = data06['DATE'].dt.strftime('%Y-%m')
data06.set_index('DATE', inplace=True)

# Assuming expansion_dates is a list of datetime objects representing the expansion dates
expansion_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in expansion] 
expansion_dates

# expansion 컬럼 추가
data06['expansion'] = data06.index.map(lambda x: 1 if x in expansion_dates else 0)

# expansion 국면만 슬라이싱
data06 = data06[data06['expansion'] == 1]
data06

# Selecting relevant columns for the regression
X = data06[['credit03', 'term03', 'equity01']]
y = data06['IEF'] - data06['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model06 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model06.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     10.38
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           6.32e-06
Time:                        23:11:40   Log-Likelihood:                 257.06
No. Observations:                  93   AIC:                            -506.1
Df Residuals:                      89   BIC:                            -496.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0352      0.007     -5.409      0.000      -0.048      -0.022
credit03      17.9464      7.238      2.479      0.015       3.564      32.328
term03       -12.9553      9.163     -1.414      0.161     -31.161       5.251
equity01      -0.0552      0.078     -0.712      0.479      -0.209       0.099
==============================================================================
Omnibus:                        6.017   Durbin-Watson:                   1.648
Prob(Omnibus):                  0.049   Jarque-Bera (JB):                5.568
Skew:                          -0.590   Prob(JB):                       0.0618
Kurtosis:                       3.215   Cond. No.                     7.19e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.19e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Slowdown - SPY

In [72]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data07 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data07 = data07.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data07 = data07.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data07 = data07.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data07 = data07.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data07 = data07.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data07 = data07.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data07 = data07.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data07 = data07.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data07.columns = ['DATE', 'SPY', 'rf', 'credit03', 'term03', 'equity01']
data07['DATE'] = pd.to_datetime(data07['DATE'])
data07['DATE'] = data07['DATE'].dt.strftime('%Y-%m')
data07.set_index('DATE', inplace=True)

# Assuming slowdown_dates is a list of datetime objects representing the slowdown dates
slowdown_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in slowdown] 
slowdown_dates

# slowdown 컬럼 추가
data07['slowdown'] = data07.index.map(lambda x: 1 if x in slowdown_dates else 0)

# slowdown 국면만 슬라이싱
data07 = data07[data07['slowdown'] == 1]
data07

# Selecting relevant columns for the regression
X = data07[['credit03', 'term03', 'equity01']]
y = data07['SPY'] - data07['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model07 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model07.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     270.1
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           2.97e-24
Time:                        23:11:40   Log-Likelihood:                 139.11
No. Observations:                  39   AIC:                            -270.2
Df Residuals:                      35   BIC:                            -263.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0229      0.005     -4.330      0.000      -0.034      -0.012
credit03      -3.0528      7.243     -0.421      0.676     -17.757      11.651
term03        17.6033     10.005      1.759      0.087      -2.708      37.914
equity01       0.9848      0.035     28.160      0.000       0.914       1.056
==============================================================================
Omnibus:                        0.245   Durbin-Watson:                   0.663
Prob(Omnibus):                  0.885   Jarque-Bera (JB):                0.436
Skew:                           0.107   Prob(JB):                        0.804
Kurtosis:                       2.528   Cond. No.                     1.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Slowdown - LQD

In [73]:
# 데이터 병합
etf_month_LQD = etf[['DATE', 'LQD']]
data08 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data08 = data08.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data08 = data08.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data08 = data08.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data08 = data08.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data08 = data08.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data08 = data08.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data08 = data08.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data08 = data08.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data08.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data08['DATE'] = pd.to_datetime(data08['DATE'])
data08['DATE'] = data08['DATE'].dt.strftime('%Y-%m')
data08.set_index('DATE', inplace=True)

# Assuming slowdown_dates is a list of datetime objects representing the slowdown dates
slowdown_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in slowdown] 
slowdown_dates

# slowdown 컬럼 추가
data08['slowdown'] = data08.index.map(lambda x: 1 if x in slowdown_dates else 0)

# slowdown 국면만 슬라이싱
data08 = data08[data08['slowdown'] == 1]
data08

# Selecting relevant columns for the regression
X = data08[['credit03', 'term03', 'equity01']]
y = data08['LQD'] - data08['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model08 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model08.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     6.016
Date:                Thu, 23 Nov 2023   Prob (F-statistic):            0.00204
Time:                        23:11:40   Log-Likelihood:                 110.85
No. Observations:                  39   AIC:                            -213.7
Df Residuals:                      35   BIC:                            -207.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0476      0.011     -4.357      0.000      -0.070      -0.025
credit03      27.6287     14.951      1.848      0.073      -2.723      57.981
term03       -26.1511     20.652     -1.266      0.214     -68.077      15.775
equity01      -0.0258      0.072     -0.358      0.723      -0.172       0.121
==============================================================================
Omnibus:                        6.606   Durbin-Watson:                   1.913
Prob(Omnibus):                  0.037   Jarque-Bera (JB):                5.262
Skew:                           0.832   Prob(JB):                       0.0720
Kurtosis:                       3.687   Cond. No.                     1.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Slowdown - IEF

In [74]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data09 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data09 = data09.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data09 = data09.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data09 = data09.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data09 = data09.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data09 = data09.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data09 = data09.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data09 = data09.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data09 = data09.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data09.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data09['DATE'] = pd.to_datetime(data09['DATE'])
data09['DATE'] = data09['DATE'].dt.strftime('%Y-%m')
data09.set_index('DATE', inplace=True)

# Assuming slowdown_dates is a list of datetime objects representing the slowdown dates
slowdown_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in slowdown] 
slowdown_dates

# slowdown 컬럼 추가
data09['slowdown'] = data09.index.map(lambda x: 1 if x in slowdown_dates else 0)

# slowdown 국면만 슬라이싱
data09 = data09[data09['slowdown'] == 1]
data09

# Selecting relevant columns for the regression
X = data09[['credit03', 'term03', 'equity01']]
y = data09['IEF'] - data09['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model09 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model09.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     8.869
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           0.000165
Time:                        23:11:41   Log-Likelihood:                 109.55
No. Observations:                  39   AIC:                            -211.1
Df Residuals:                      35   BIC:                            -204.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0427      0.011     -3.781      0.001      -0.066      -0.020
credit03      27.1946     15.456      1.760      0.087      -4.182      58.571
term03       -27.9235     21.349     -1.308      0.199     -71.265      15.418
equity01      -0.2455      0.075     -3.290      0.002      -0.397      -0.094
==============================================================================
Omnibus:                        2.833   Durbin-Watson:                   1.797
Prob(Omnibus):                  0.243   Jarque-Bera (JB):                1.913
Skew:                           0.528   Prob(JB):                        0.384
Kurtosis:                       3.250   Cond. No.                     1.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Contraction - SPY

In [75]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data10 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data10 = data10.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data10 = data10.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data10 = data10.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data10 = data10.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data10 = data10.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data10 = data10.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data10 = data10.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data10 = data10.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data10.columns = ['DATE', 'SPY', 'rf', 'credit03', 'term03', 'equity01']
data10['DATE'] = pd.to_datetime(data10['DATE'])
data10['DATE'] = data10['DATE'].dt.strftime('%Y-%m')
data10.set_index('DATE', inplace=True)

# Assuming contraction_dates is a list of datetime objects representing the contraction dates
contraction_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in contraction] 
contraction_dates

# contraction 컬럼 추가
data10['contraction'] = data10.index.map(lambda x: 1 if x in contraction_dates else 0)

# contraction 국면만 슬라이싱
data10 = data10[data10['contraction'] == 1]
data10

# Selecting relevant columns for the regression
X = data10[['credit03', 'term03', 'equity01']]
y = data10['SPY'] - data10['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model10 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model10.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     1934.
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           8.07e-40
Time:                        23:11:41   Log-Likelihood:                 154.36
No. Observations:                  40   AIC:                            -300.7
Df Residuals:                      36   BIC:                            -294.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0137      0.005     -2.830      0.008      -0.024      -0.004
credit03       7.0521      3.842      1.835      0.075      -0.740      14.845
term03        -5.5297      5.005     -1.105      0.277     -15.681       4.622
equity01       1.0132      0.014     74.805      0.000       0.986       1.041
==============================================================================
Omnibus:                        7.470   Durbin-Watson:                   1.066
Prob(Omnibus):                  0.024   Jarque-Bera (JB):                7.049
Skew:                          -1.027   Prob(JB):                       0.0295
Kurtosis:                       3.100   Cond. No.                     7.39e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.39e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Contraction - LQD

In [76]:
# 데이터 병합
etf_month_LQD = etf[['DATE', 'LQD']]
data11 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data11 = data11.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data11 = data11.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data11 = data11.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data11 = data11.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data11 = data11.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data11 = data11.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data11 = data11.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data11 = data11.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data11.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data11['DATE'] = pd.to_datetime(data11['DATE'])
data11['DATE'] = data11['DATE'].dt.strftime('%Y-%m')
data11.set_index('DATE', inplace=True)

# Assuming contraction_dates is a list of datetime objects representing the contraction dates
contraction_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in contraction] 
contraction_dates

# contraction 컬럼 추가
data11['contraction'] = data11.index.map(lambda x: 1 if x in contraction_dates else 0)

# contraction 국면만 슬라이싱
data11 = data11[data11['contraction'] == 1]
data11

# Selecting relevant columns for the regression
X = data11[['credit03', 'term03', 'equity01']]
y = data11['LQD'] - data11['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model11 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     3.854
Date:                Thu, 23 Nov 2023   Prob (F-statistic):             0.0173
Time:                        23:11:41   Log-Likelihood:                 80.756
No. Observations:                  40   AIC:                            -153.5
Df Residuals:                      36   BIC:                            -146.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0289      0.030     -0.947      0.350      -0.091       0.033
credit03      25.8028     24.194      1.066      0.293     -23.266      74.871
term03       -29.2789     31.519     -0.929      0.359     -93.203      34.645
equity01       0.2778      0.085      3.257      0.002       0.105       0.451
==============================================================================
Omnibus:                       10.895   Durbin-Watson:                   1.744
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               27.452
Skew:                           0.276   Prob(JB):                     1.09e-06
Kurtosis:                       7.021   Cond. No.                     7.39e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.39e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Contraction - IEF

In [77]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data12 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data12 = data12.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data12 = data12.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data12 = data12.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data12 = data12.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data12 = data12.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data12 = data12.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data12 = data12.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data12 = data12.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data12.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data12['DATE'] = pd.to_datetime(data12['DATE'])
data12['DATE'] = data12['DATE'].dt.strftime('%Y-%m')
data12.set_index('DATE', inplace=True)

# Assuming contraction_dates is a list of datetime objects representing the contraction dates
contraction_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in contraction] 
contraction_dates

# contraction 컬럼 추가
data12['contraction'] = data12.index.map(lambda x: 1 if x in contraction_dates else 0)

# contraction 국면만 슬라이싱
data12 = data12[data12['contraction'] == 1]
data12

# Selecting relevant columns for the regression
X = data12[['credit03', 'term03', 'equity01']]
y = data12['IEF'] - data12['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model12 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.8419
Date:                Thu, 23 Nov 2023   Prob (F-statistic):              0.480
Time:                        23:11:41   Log-Likelihood:                 96.075
No. Observations:                  40   AIC:                            -184.2
Df Residuals:                      36   BIC:                            -177.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0161      0.021     -0.773      0.445      -0.058       0.026
credit03      17.0195     16.496      1.032      0.309     -16.437      50.476
term03       -19.8269     21.491     -0.923      0.362     -63.412      23.758
equity01      -0.0555      0.058     -0.955      0.346      -0.173       0.062
==============================================================================
Omnibus:                        1.088   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.580   Jarque-Bera (JB):                0.329
Skew:                           0.025   Prob(JB):                        0.848
Kurtosis:                       3.441   Cond. No.                     7.39e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.39e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Buy and Hold - SPY

In [78]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data13 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data13 = data13.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data13 = data13.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data13 = data13.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data13 = data13.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data13 = data13.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data13 = data13.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data13 = data13.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data13 = data13.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data13.columns = ['DATE', 'SPY', 'rf', 'credit03', 'term03', 'equity01']
data13['DATE'] = pd.to_datetime(data13['DATE'])
data13['DATE'] = data13['DATE'].dt.strftime('%Y-%m')
data13.set_index('DATE', inplace=True)

# Assuming bh_dates is a list of datetime objects representing the bh dates
bh_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in bh] 
bh_dates

# bh 컬럼 추가
data13['bh'] = data13.index.map(lambda x: 1 if x in bh_dates else 0)

# bh 국면만 슬라이싱
data13 = data13[data13['bh'] == 1]
data13

# Selecting relevant columns for the regression
X = data13[['credit03', 'term03', 'equity01']]
y = data13['SPY'] - data13['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model13 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model13.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     3954.
Date:                Thu, 23 Nov 2023   Prob (F-statistic):          1.11e-191
Time:                        23:11:41   Log-Likelihood:                 839.21
No. Observations:                 225   AIC:                            -1670.
Df Residuals:                     221   BIC:                            -1657.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0292      0.002    -19.457      0.000      -0.032      -0.026
credit03      14.9900      1.403     10.685      0.000      12.225      17.755
term03       -10.7027      1.778     -6.020      0.000     -14.206      -7.199
equity01       1.0286      0.010    107.628      0.000       1.010       1.047
==============================================================================
Omnibus:                       18.975   Durbin-Watson:                   0.543
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.432
Skew:                          -0.681   Prob(JB):                     2.22e-05
Kurtosis:                       3.657   Cond. No.                     5.76e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.76e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Buy and Hold - LQD

In [79]:
# 데이터 병합
etf_month_LQD = etf[['DATE', 'LQD']]
data14 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data14 = data14.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data14 = data14.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data14 = data14.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data14 = data14.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data14 = data14.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data14 = data14.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data14 = data14.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data14 = data14.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data14.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data14['DATE'] = pd.to_datetime(data14['DATE'])
data14['DATE'] = data14['DATE'].dt.strftime('%Y-%m')
data14.set_index('DATE', inplace=True)

# Assuming bh_dates is a list of datetime objects representing the bh dates
bh_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in bh] 
bh_dates

# bh 컬럼 추가
data14['bh'] = data14.index.map(lambda x: 1 if x in bh_dates else 0)

# bh 국면만 슬라이싱
data14 = data14[data14['bh'] == 1]
data14

# Selecting relevant columns for the regression
X = data14[['credit03', 'term03', 'equity01']]
y = data14['LQD'] - data14['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model14 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model14.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     29.05
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           7.12e-16
Time:                        23:11:41   Log-Likelihood:                 557.58
No. Observations:                 225   AIC:                            -1107.
Df Residuals:                     221   BIC:                            -1093.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0405      0.005     -7.728      0.000      -0.051      -0.030
credit03      26.4728      4.905      5.397      0.000      16.806      36.139
term03       -24.6009      6.216     -3.958      0.000     -36.850     -12.352
equity01       0.1964      0.033      5.878      0.000       0.131       0.262
==============================================================================
Omnibus:                       42.098   Durbin-Watson:                   1.878
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              476.690
Skew:                          -0.049   Prob(JB):                    3.08e-104
Kurtosis:                      10.130   Cond. No.                     5.76e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.76e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Buy and Hold - IEF

In [80]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data15 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data15 = data15.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data15 = data15.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data15 = data15.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data15 = data15.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data15 = data15.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data15 = data15.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data15 = data15.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data15 = data15.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data15.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data15['DATE'] = pd.to_datetime(data15['DATE'])
data15['DATE'] = data15['DATE'].dt.strftime('%Y-%m')
data15.set_index('DATE', inplace=True)

# Assuming bh_dates is a list of datetime objects representing the bh dates
bh_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in bh] 
bh_dates

# bh 컬럼 추가
data15['bh'] = data15.index.map(lambda x: 1 if x in bh_dates else 0)

# bh 국면만 슬라이싱
data15 = data15[data15['bh'] == 1]
data15

# Selecting relevant columns for the regression
X = data15[['credit03', 'term03', 'equity01']]
y = data15['IEF'] - data15['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model15 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model15.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.250
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     24.51
Date:                Thu, 23 Nov 2023   Prob (F-statistic):           9.95e-14
Time:                        23:11:41   Log-Likelihood:                 589.69
No. Observations:                 225   AIC:                            -1171.
Df Residuals:                     221   BIC:                            -1158.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0358      0.005     -7.881      0.000      -0.045      -0.027
credit03      23.4456      4.253      5.513      0.000      15.065      31.827
term03       -21.6025      5.389     -4.009      0.000     -32.223     -10.982
equity01      -0.0849      0.029     -2.931      0.004      -0.142      -0.028
==============================================================================
Omnibus:                       10.065   Durbin-Watson:                   1.845
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               11.416
Skew:                          -0.392   Prob(JB):                      0.00332
Kurtosis:                       3.777   Cond. No.                     5.76e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.76e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### OLS result

In [81]:
co1 = model01.params
co2 = model02.params
co3 = model03.params
co4 = model04.params
co5 = model05.params
co6 = model06.params
co7 = model07.params
co8 = model08.params
co9 = model09.params
co10 = model10.params
co11 = model11.params
co12 = model12.params
co13 = model13.params
co14 = model14.params
co15 = model15.params

df1 = pd.DataFrame(co1, columns=['Coefficient'])
df2 = pd.DataFrame(co2, columns=['Coefficient'])
df3 = pd.DataFrame(co3, columns=['Coefficient'])
df4 = pd.DataFrame(co4, columns=['Coefficient'])
df5 = pd.DataFrame(co5, columns=['Coefficient'])
df6 = pd.DataFrame(co6, columns=['Coefficient'])
df7 = pd.DataFrame(co7, columns=['Coefficient'])
df8 = pd.DataFrame(co8, columns=['Coefficient'])
df9 = pd.DataFrame(co9, columns=['Coefficient'])
df10 = pd.DataFrame(co10, columns=['Coefficient'])
df11 = pd.DataFrame(co11, columns=['Coefficient'])
df12 = pd.DataFrame(co12, columns=['Coefficient'])
df13 = pd.DataFrame(co13, columns=['Coefficient'])
df14 = pd.DataFrame(co14, columns=['Coefficient'])
df15 = pd.DataFrame(co15, columns=['Coefficient'])


ols_result_coef = pd.concat([df1,df4,df7,df10,df13,df2,df5,df8,df11,df14,df3,df6,df9,df12,df15], axis=1)

ols_result_coef.columns = [['recovery-SPY', 'expansion-SPY','slowdown-SPY','contraction-SPY','buy_and_hold-SPY',
                            'recovery-LQD','expansion-LQD','slowdown-LQD','contraction-LQD','buy_and_hold-LQD',
                            'recovery-IEF',  'expansion-IEF','slowdown-IEF', 'contraction-IEF', 'buy_and_hold-IEF']]

In [82]:
ols_result_coef

,recovery-SPY,expansion-SPY,slowdown-SPY,contraction-SPY,buy_and_hold-SPY,recovery-LQD,expansion-LQD,slowdown-LQD,contraction-LQD,buy_and_hold-LQD,recovery-IEF,expansion-IEF,slowdown-IEF,contraction-IEF,buy_and_hold-IEF
const,-0.023599,-0.033440,-0.022929,-0.013690,-0.029188,-0.039478,-0.032837,-0.047624,-0.028855,-0.040531,-0.023269,-0.035229,-0.042718,-0.016056,-0.035837
credit03,14.004491,19.492454,-3.052820,7.052141,14.990023,29.745630,15.109540,27.628660,25.802809,26.472786,14.752066,17.946363,27.194599,17.019543,23.445604
term03,-11.337325,-15.359060,17.603252,-5.529662,-10.702716,-28.301957,-9.569131,-26.151098,-29.278920,-24.600885,-12.053878,-12.955286,-27.923484,-19.826864,-21.602532
equity01,0.994203,1.031308,0.984757,1.013235,1.028608,0.124723,0.102093,-0.025820,0.277811,0.196422,-0.121329,-0.055206,-0.245535,-0.055518,-0.084911


### 시각화 (Panel A)

In [83]:
# data = {
#     'Recovery': [5, 3, 2, 4, 6],  # 회복 국면에서의 수익률(베타계수)
#     'Expansion': [7, 5, 3, 6, 8],  # 확장 국면에서의 수익률(베타계수)
#     'Slowdown': [-2, -1, 0, -1, -2],  # 둔화 국면에서의 수익률(베타계수)
#     'Contraction': [-4, -3, -2, -3, -4],  # 수축 국면에서의 수익률(베타계수)
#     'Buy & Hold': [4, 3, 5, 4, 5]  # 매수 및 보유 국면에서의 수익률(베타계수)
# }
# asset_classes = ['SPY', 'LQD', 'IEF']
# phases = list(data.keys())

# # 데이터를 기반으로 그래프를 그립니다.
# n = len(asset_classes)
# r = np.arange(n)
# width = 0.15

# fig, ax = plt.subplots(figsize=(10, 6))

# for i, phase in enumerate(phases):
#     ax.bar(r + i*width, data[phase], width=width, label=phase)

# # 축 이름과 제목을 추가합니다.
# ax.set_xlabel('Asset Class')
# ax.set_ylabel('Performance (%)')
# ax.set_title('Risk Premia Mapping and Conditional Performance')
# ax.set_xticks(r + width * (len(phases) - 1) / 2)
# ax.set_xticklabels(asset_classes)
# ax.legend()

# # 그래프를 표시합니다.
# plt.show()

In [84]:
ols_result_coef.to_csv('./ols_result_coef.csv')

In [85]:
ols_result_coef

,recovery-SPY,expansion-SPY,slowdown-SPY,contraction-SPY,buy_and_hold-SPY,recovery-LQD,expansion-LQD,slowdown-LQD,contraction-LQD,buy_and_hold-LQD,recovery-IEF,expansion-IEF,slowdown-IEF,contraction-IEF,buy_and_hold-IEF
const,-0.023599,-0.033440,-0.022929,-0.013690,-0.029188,-0.039478,-0.032837,-0.047624,-0.028855,-0.040531,-0.023269,-0.035229,-0.042718,-0.016056,-0.035837
credit03,14.004491,19.492454,-3.052820,7.052141,14.990023,29.745630,15.109540,27.628660,25.802809,26.472786,14.752066,17.946363,27.194599,17.019543,23.445604
term03,-11.337325,-15.359060,17.603252,-5.529662,-10.702716,-28.301957,-9.569131,-26.151098,-29.278920,-24.600885,-12.053878,-12.955286,-27.923484,-19.826864,-21.602532
equity01,0.994203,1.031308,0.984757,1.013235,1.028608,0.124723,0.102093,-0.025820,0.277811,0.196422,-0.121329,-0.055206,-0.245535,-0.055518,-0.084911


In [86]:
def anuual_ret(df):
    etf = (((df.iloc[:,0]) + 1) ** 12) - 1  # etf 연 평균 수익률 -> 소수점 단위 변경
    rf = (((df.iloc[:,1]) + 1) ** 12) - 1  # rf
    credit = (((df.iloc[:,2] ) + 1) ** 12) - 1  # credit
    term = (((df.iloc[:,3] ) + 1) ** 12) - 1  # term
    equity = (((df.iloc[:,4]) + 1) ** 12) - 1  # equity
    lst = [etf.mean(), rf.mean(), credit.mean(), term.mean(), equity.mean()] 
    return lst  # 소수점 단위 값

In [87]:
# def anuual_ret(df):
#     etf = (df.iloc[:,0]).mean()  # etf 연 평균 수익률 -> 소수점 단위 변경
#     rf = (df.iloc[:,1]).mean()  # rf
#     credit = (df.iloc[:,2] ).mean()  # credit
#     term = (df.iloc[:,3] ).mean()# term
#     equity = (df.iloc[:,4]).mean()  # equity
#     lst = [etf, rf, credit, term, equity] 
#     return lst  # 소수점 단위 값

In [88]:
print('ret01 =',anuual_ret(data01))  # Recovery-SPY
print('ret02 =',anuual_ret(data02))  # Recovery-LQD
print('ret03 =',anuual_ret(data03))  # Recovery-IEF
print('ret04 =',anuual_ret(data04))  # Expansion-SPY
print('ret05 =',anuual_ret(data05))  # Expansion-LQD
print('ret06 =',anuual_ret(data06))  # Expansion-IEF
print('ret07 =',anuual_ret(data07))  # Slowdown-SPY
print('ret08 =',anuual_ret(data08))  # Slowdown-LQD
print('ret09 =',anuual_ret(data09))  # Slowdown-IEF
print('ret10 =',anuual_ret(data10))  # Contraction-SPY
print('ret11 =',anuual_ret(data11))  # Contraction-LQD
print('ret12 =',anuual_ret(data12))  # Contraction-IEF
print('ret13 =',anuual_ret(data13))  # Buy and Hold-SPY
print('ret14 =',anuual_ret(data14))  # Buy and Hold-LQD
print('ret15 =',anuual_ret(data15))  # Buy and Hold-IEF

# [etf, rf, credit, term, equity] 

ret01 = [0.47739051940075944, 0.04563651745348951, 0.03806424848649424, 0.020933472915717767, 0.41166760256144397]
ret02 = [0.16304007007373927, 0.04563651745348951, 0.03806424848649424, 0.020933472915717767, 0.41166760256144397]
ret03 = [0.05434076292219162, 0.04563651745348951, 0.03806424848649424, 0.020933472915717767, 0.41166760256144397]
ret04 = [0.2917469025960519, 0.11499050132059661, 0.03254846437839061, 0.018308586907374226, 0.22938108032823804]
ret05 = [0.04781179442972429, 0.11499050132059661, 0.03254846437839061, 0.018308586907374226, 0.22938108032823804]
ret06 = [0.022053784000393303, 0.11499050132059661, 0.03254846437839061, 0.018308586907374226, 0.22938108032823804]
ret07 = [-0.048072104134659704, 0.2335030376043484, 0.023492815414992253, 0.010997448094368431, -0.09297089601407157]
ret08 = [0.0031134475521303654, 0.2335030376043484, 0.023492815414992253, 0.010997448094368431, -0.09297089601407157]
ret09 = [0.07748043226695259, 0.2335030376043484, 0.023492815414992253, 0.

In [89]:
ret01 = anuual_ret(data01)  # Recovery-SPY
ret02 = anuual_ret(data02)  # Recovery-LQD
ret03 = anuual_ret(data03)  # Recovery-IEF
ret04 = anuual_ret(data04)  # Expansion-SPY
ret05 = anuual_ret(data05)  # Expansion-LQD
ret06 = anuual_ret(data06)  # Expansion-IEF
ret07 = anuual_ret(data07)  # Slowdown-SPY
ret08 = anuual_ret(data08)  # Slowdown-LQD
ret09 = anuual_ret(data09)  # Slowdown-IEF
ret10 = anuual_ret(data10)  # Contraction-SPY
ret11 = anuual_ret(data11)  # Contraction-LQD
ret12 = anuual_ret(data12)  # Contraction-IEF
ret13 = anuual_ret(data13)  # Buy and Hold-SPY
ret14 = anuual_ret(data14)  # Buy and Hold-LQD
ret15 = anuual_ret(data15)  # Buy and Hold-IEF

In [90]:
rets = [ret01, ret04, ret07, ret10, ret13, ret02, ret05, ret08, ret11, ret14, ret03, ret06, ret09, ret12, ret15]
annual_return = pd.DataFrame(rets, columns=['etf', 'rf', 'credit', 'term', 'equity'])
annual_return.to_csv('./annual_return.csv')

In [91]:
ols_result_coef

,recovery-SPY,expansion-SPY,slowdown-SPY,contraction-SPY,buy_and_hold-SPY,recovery-LQD,expansion-LQD,slowdown-LQD,contraction-LQD,buy_and_hold-LQD,recovery-IEF,expansion-IEF,slowdown-IEF,contraction-IEF,buy_and_hold-IEF
const,-0.023599,-0.033440,-0.022929,-0.013690,-0.029188,-0.039478,-0.032837,-0.047624,-0.028855,-0.040531,-0.023269,-0.035229,-0.042718,-0.016056,-0.035837
credit03,14.004491,19.492454,-3.052820,7.052141,14.990023,29.745630,15.109540,27.628660,25.802809,26.472786,14.752066,17.946363,27.194599,17.019543,23.445604
term03,-11.337325,-15.359060,17.603252,-5.529662,-10.702716,-28.301957,-9.569131,-26.151098,-29.278920,-24.600885,-12.053878,-12.955286,-27.923484,-19.826864,-21.602532
equity01,0.994203,1.031308,0.984757,1.013235,1.028608,0.124723,0.102093,-0.025820,0.277811,0.196422,-0.121329,-0.055206,-0.245535,-0.055518,-0.084911


In [92]:
ret13

[0.2432068725065515,
 0.10856705972679556,
 0.0328618141046055,
 0.01733961433883553,
 0.18706465791616703]

In [93]:
# y(SPY) 
re_SPY = ols_result_coef.loc['credit03','recovery-SPY']      * ret01[2] + ols_result_coef.loc['term03', 'recovery-SPY']     * ret01[3] + ols_result_coef.loc['equity01', 'recovery-SPY']     * ret01[4]
ex_SPY = ols_result_coef.loc['credit03','expansion-SPY']     * ret04[2] + ols_result_coef.loc['term03', 'expansion-SPY']    * ret04[3] + ols_result_coef.loc['equity01', 'expansion-SPY']    * ret04[4]
sl_SPY = ols_result_coef.loc['credit03','slowdown-SPY']      * ret07[2] + ols_result_coef.loc['term03', 'slowdown-SPY']     * ret07[3] + ols_result_coef.loc['equity01', 'slowdown-SPY']     * ret07[4]
con_SPY =ols_result_coef.loc['credit03','contraction-SPY']   * ret10[2] + ols_result_coef.loc['term03', 'contraction-SPY']  * ret10[3] + ols_result_coef.loc['equity01', 'contraction-SPY']  * ret10[4]
bh_SPY = ols_result_coef.loc['const','buy_and_hold-SPY'] + ols_result_coef.loc['credit03','buy_and_hold-SPY']  * ret13[2] + ols_result_coef.loc['term03', 'buy_and_hold-SPY'] * ret13[3] + ols_result_coef.loc['equity01', 'buy_and_hold-SPY'] * ret13[4]

In [94]:
# y(LQD) 
re_LQD =  ols_result_coef.loc['credit03','recovery-LQD']     * ret02[2] + ols_result_coef.loc['term03', 'recovery-LQD']     * ret02[3] + ols_result_coef.loc['equity01', 'recovery-LQD']     * ret02[4]
ex_LQD =  ols_result_coef.loc['credit03','expansion-LQD']    * ret05[2] + ols_result_coef.loc['term03', 'expansion-LQD']    * ret05[3] + ols_result_coef.loc['equity01', 'expansion-LQD']    * ret05[4]
sl_LQD =  ols_result_coef.loc['credit03','slowdown-LQD']     * ret08[2] + ols_result_coef.loc['term03', 'slowdown-LQD']     * ret08[3] + ols_result_coef.loc['equity01', 'slowdown-LQD']     * ret08[4]
con_LQD = ols_result_coef.loc['credit03','contraction-LQD']  * ret11[2] + ols_result_coef.loc['term03', 'contraction-LQD']  * ret11[3] + ols_result_coef.loc['equity01', 'contraction-LQD']  * ret11[4]
bh_LQD =  ols_result_coef.loc['const','buy_and_hold-LQD'] + ols_result_coef.loc['credit03','buy_and_hold-LQD'] * ret14[2] + ols_result_coef.loc['term03', 'buy_and_hold-LQD'] * ret14[3] + ols_result_coef.loc['equity01', 'buy_and_hold-LQD'] * ret14[4]

In [95]:
# y(IEF) 
re_IEF = ols_result_coef.loc['credit03','recovery-IEF']     * ret03[2] + ols_result_coef.loc['term03', 'recovery-IEF']     * ret03[3] + ols_result_coef.loc['equity01', 'recovery-IEF']     * ret03[4]
ex_IEF =  ols_result_coef.loc['credit03','expansion-IEF']    * ret06[2] + ols_result_coef.loc['term03', 'expansion-IEF']    * ret06[3] + ols_result_coef.loc['equity01', 'expansion-IEF']    * ret06[4]
sl_IEF = ols_result_coef.loc['credit03','slowdown-IEF']     * ret09[2] + ols_result_coef.loc['term03', 'slowdown-IEF']     * ret09[3] + ols_result_coef.loc['equity01', 'slowdown-IEF']     * ret09[4]
con_IEF = ols_result_coef.loc['credit03','contraction-IEF']  * ret12[2] + ols_result_coef.loc['term03', 'contraction-IEF']  * ret12[3] + ols_result_coef.loc['equity01', 'contraction-IEF']  * ret12[4]
bh_IEF =  ols_result_coef.loc['const','buy_and_hold-IEF'] + ols_result_coef.loc['credit03','buy_and_hold-IEF'] * ret15[2] + ols_result_coef.loc['term03', 'buy_and_hold-IEF'] * ret15[3] + ols_result_coef.loc['equity01', 'buy_and_hold-IEF'] * ret15[4]

In [96]:
print(bh_SPY)
print(bh_LQD)
print(bh_IEF)

buy_and_hold-SPY    0.470247
dtype: float64
buy_and_hold-LQD    0.439586
dtype: float64
buy_and_hold-IEF    0.344164
dtype: float64


In [97]:
# data = {
#     'SPY' : [re_SPY, ex_SPY, sl_SPY, con_SPY, bh_SPY], # SPY에서의 국면별 수익률
#     'LQD' : [re_LQD, ex_LQD, sl_LQD, con_LQD, bh_LQD], # LQD에서의 국면별 수익률
#     'IEF' : [re_IEF, ex_IEF, sl_IEF, con_IEF, bh_IEF], # IEF에서의 국면별 수익률
# }

# regime_classes = ['recovery', 'expansion', 'slowdown', 'contraction', 'buy & hold']
# phases = list.data(keys())

# # 데이터 기반 시각화
# n = len(regime_classes)
# r = np.arange(n)
# width = 0.15

# fig, ax = plt.subplots(figsize=(10, 6))

# for i, phase in enumerate(phases):
#     ax.bar(r + i*width, data[phase], width=width, label=phase)

# # 축 이름 및 제목 설정
# ax.set_xlabel('Asset Class')
# ax.set_ylabel('Performance (%)')
# ax.set_title('Risk Premia Mapping and Conditional Performance')
# ax.set_xticks(r + width * (len(phases) - 1) / 2)
# ax.set_xticklabels(asset_classes)
# ax.legend()

# plt.show()

In [99]:
# float(ols_result_coef.loc['rf', 'recovery-SPY'] * ret01[1]) > 0

In [100]:
# contributions = {
#     'Risk-Free Rate': ols_result_coef.loc['rf', 'recovery-SPY'] * ret01[1],
#     'Credit Premium': ols_result_coef.loc['credit03', 'recovery-SPY'] * ret01[2],
#     'Term Premium': ols_result_coef.loc['term03', 'recovery-SPY'] * ret01[3],
#     'Equity Premium': ols_result_coef.loc['equity01', 'recovery-SPY'] * ret01[4]
# }

# # Plotting the stacked bar chart
# fig, ax = plt.subplots()

# # Initialize the bottom parameters for positive and negative stacks
# bottom_positive = 0
# bottom_negative = 0
# for name, value in contributions.items():
#     if float(value) > 0:
#         # Stack positive values on top of each other
#         ax.bar('recovery-SPY', value, label=name, bottom=bottom_positive)
#         bottom_positive += value
#     else:
#         # Stack negative values below each other
#         ax.bar('recovery-SPY', value, label=name, bottom=bottom_negative)
#         bottom_negative += value  # Since value is negative, this subtracts it

# ax.set_ylabel('Contribution to SPY Recovery Premia')
# ax.legend()

# plt.show()

In [102]:
# fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

# # Titles for each subplot
# titles = ['SPY', 'LQD', 'IEF']

# # Plot each bar chart in a separate subplot with synthetic data
# for i, ax in enumerate(axes.flatten()):
#     ax.bar('recovery-{}'.format(titles[i]), contributions['Risk-Free Rate'], label='Risk-Free Rate', width=0.1, color='green')
#     ax.bar('recovery-{}'.format(titles[i]), contributions['Credit Premium'], label='Credit Premium', width=0.1, color='blue', bottom=contributions['Risk-Free Rate'])
#     ax.bar('recovery-{}'.format(titles[i]), contributions['Term Premium'], label='Term Premium', width=0.1, color='orange', bottom=contributions['Risk-Free Rate'] + contributions['Credit Premium'])
#     ax.bar('recovery-{}'.format(titles[i]), contributions['Equity Premium'], label='Equity Premium', width=0.1, color='red', bottom=contributions['Risk-Free Rate'] + contributions['Credit Premium'] + contributions['Term Premium'])
#     ax.set_title(titles[i])

# # Set common labels
# fig.text(0.5, 0.04, 'Economic Indicator', ha='center', va='center')
# fig.text(0.06, 0.5, 'Contribution to Recovery Premia', ha='center', va='center', rotation='vertical')

# # Display the legend
# axes[0].legend(loc='upper left')
# plt.tight_layout()
# plt.show()